In [1]:
import Pkg
#Pkg.add("MathOptInterface")
#Pkg.add("GLPK")
Pkg.activate(@__DIR__)
Pkg.instantiate()

Activating environment at `C:\Users\mmart\Desktop\TFM\Julia\notebooks\Project.toml`


In [2]:
using JuMP
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver

In [3]:
include("../src/read_data.jl")
include("../JuMP/main_JuMP.jl")

H0 = 20

Constants(5, 5, 5, 2, 800, 1, 1)

In [4]:
PP=[p for p in 1:P]
QQ=[q for q in 1:Q]
T=P+1

function travel_time(current_bay::Int, target_bay::Int, CTS::Constants)
    return(abs(current_bay - target_bay)*CTS.tt)
end

#create t_tasks and bj_dict
t_task = Dict{Int, Int}()
bj_dict = Dict{Int, Int}()
bj_init = Dict{Int, Int}()
for p in PP
    t_task[p] = tasks_by_position[p][1].t
    bj_dict[p] = bj[p]
end

t_task[0] = 0
t_task[T] = 0
bj_dict[0] = 0
bj_dict[T] = 0
bj_init[1] = 1
bj_init[2] = 4
bj_dict

Dict{Int64,Int64} with 7 entries:
  0 => 0
  4 => 3
  2 => 1
  3 => 2
  5 => 5
  6 => 0
  1 => 1

In [5]:
model = Model(optimizer_with_attributes(GLPK.Optimizer));
# Old syntax: model = Model(solver=GLPKSolverLP(msg_lev = 4)))

In [6]:
#if positions p, s are performed consecutively by crane q
@variable(model, x[p=0:P+1, s=0:P+1, q=1:Q], Bin)
#if positions p, is filled with container i by crane q
#@variable(model, w[p=0:T, i=1:C, q=1:Q], Bin)
#if position s starts after the completion time of task p
@variable(model, z[p=1:P, s=1:P], Bin)

#completion time of task p
@variable(model, t_load[p=1:P], lower_bound=0, upper_bound=CTS.H)
#completion time of crane q
@variable(model, t_crane[q=1:Q], lower_bound=0, upper_bound=CTS.H)
#makespan
@variable(model, makespan, lower_bound=0, upper_bound=CTS.H)


#objective function
#@objective(model, Min, 0.8*makespan + 0.2*sum(t_crane[q] for q in QQ))
@objective(model, Min, makespan)

makespan

In [7]:
@constraint(model, [p=0:P+1, q=1:Q], x[p,p,q] == 0)
@constraint(model, [p=1:P], z[p,p] == 0)

#@constraint(model, [p=1:P], t_task[p] == sum(sum(2*task_times[p,i]*w[p,i,q] for q in subset_crane_pos(CTS, p, bj)) for i in subset_pos(CC, tasks_by_position, p)))

5-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}},ScalarShape},1}:
 z[1,1] == 0.0
 z[2,2] == 0.0
 z[3,3] == 0.0
 z[4,4] == 0.0
 z[5,5] == 0.0

In [8]:
#@constraint(model, [i=1:C, q=1:Q], w[0,i,q] == 0)
#@constraint(model, [i=1:C, q=1:Q], w[T,i,q] == 0)

In [9]:
@constraint(model, [q=1:Q], t_crane[q] <= makespan)

2-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 t_crane[1] - makespan <= 0.0
 t_crane[2] - makespan <= 0.0

In [10]:
@constraint(model, [q=1:Q], sum(x[0,s,q] for s in PP) == 1)
@constraint(model, [q=1:Q], sum(x[p,T,q] for p in PP) == 1)

2-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}},ScalarShape},1}:
 x[1,6,1] + x[2,6,1] + x[3,6,1] + x[4,6,1] + x[5,6,1] == 1.0
 x[1,6,2] + x[2,6,2] + x[3,6,2] + x[4,6,2] + x[5,6,2] == 1.0

In [11]:
@constraint(model, [p=1:P], sum(sum(x[p,s,q] for s in PP) for q in QQ) == 1)

@constraint(model, [p=1:P, q=1:Q], sum(x[s,p,q] for s in PP) - sum(x[p,s,q] for s in PP) == 0)

5×2 Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}},ScalarShape},2}:
 x[2,1,1] + x[3,1,1] + x[4,1,1] + x[5,1,1] - x[1,2,1] - x[1,3,1] - x[1,4,1] - x[1,5,1] == 0.0  …  x[2,1,2] + x[3,1,2] + x[4,1,2] + x[5,1,2] - x[1,2,2] - x[1,3,2] - x[1,4,2] - x[1,5,2] == 0.0
 x[1,2,1] + x[3,2,1] + x[4,2,1] + x[5,2,1] - x[2,1,1] - x[2,3,1] - x[2,4,1] - x[2,5,1] == 0.0     x[1,2,2] + x[3,2,2] + x[4,2,2] + x[5,2,2] - x[2,1,2] - x[2,3,2] - x[2,4,2] - x[2,5,2] == 0.0
 x[1,3,1] + x[2,3,1] + x[4,3,1] + x[5,3,1] - x[3,1,1] - x[3,2,1] - x[3,4,1] - x[3,5,1] == 0.0     x[1,3,2] + x[2,3,2] + x[4,3,2] + x[5,3,2] - x[3,1,2] - x[3,2,2] - x[3,4,2] - x[3,5,2] == 0.0
 x[1,4,1] + x[2,4,1] + x[3,4,1] + x[5,4,1] - x[4,1,1] - x[4,2,1] - x[4,3,1] - x[4,5,1] == 0.0     x[1,4,2] + x[2,4,2] + x[3,4,2] + x[5,4,2] - x[4,1,2] - x[4,2,2] - x[4,3,2] - x[4,5,2] == 0.0
 x[1,5,1] + x[2,5,1] + x[3,5,1] + x[4,5,1] - x[5,1,1] - x[5,2,1] - x[5,3,1

In [12]:
for s in PP
    for p in prec[s]
        @constraint(model, t_load[p] + t_task[s] - t_load[s] <= 0)
        @constraint(model, z[p,s] + z[s,p] == 1)
    end
end

In [13]:
for p in PP
    for s in PP
        if p != s
            for q in QQ
                @constraint(model, t_load[p] + travel_time(bj_dict[p], bj_dict[s], CTS) + t_task[s] - t_load[s] <= CTS.H*(1-x[p,s,q]))
                @constraint(model, t_load[p] + t_task[s] - t_load[s] <= CTS.H*(1-z[p,s]))
            end
        end
    end
end

In [14]:
for p in PP
    for s in PP
        if p != s && bj_dict[p] < bj_dict[s]
            for q in QQ
                @constraint(model, sum(sum(x[u,s,v] for u in PP) for v in collect(1:q)) - sum(sum(x[u,p,v] for u in PP) for v in collect(1:q)) <= CTS.H*(z[p,s] + z[s,p]))
            end
        end
    end
end

In [15]:
#@constraint(model, [p=1:P, q=1:Q], t_load[p] + travel_time(p, bj_start_end[1, q], CTS) - t_crane[q] <= CTS.H*(1-x[p,T,q]))
@constraint(model, [p=1:P, q=1:Q], t_load[p] - t_crane[q] <= CTS.H*(1-x[p,T,q]))

@constraint(model, [s=1:P, q=1:Q], travel_time(bj_init[q], s, CTS) + t_task[s] - t_load[s] <= CTS.H*(1-x[0,s,q]))
#@constraint(model, [s=1:P, q=1:Q], t_task[s] - t_load[s] <= CTS.H*(1-x[0,s,q]))

5×2 Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},2}:
 -t_load[1] + 800 x[0,1,1] <= 778.0  -t_load[1] + 800 x[0,1,2] <= 775.0
 -t_load[2] + 800 x[0,2,1] <= 753.0  -t_load[2] + 800 x[0,2,2] <= 752.0
 -t_load[3] + 800 x[0,3,1] <= 790.0  -t_load[3] + 800 x[0,3,2] <= 791.0
 -t_load[4] + 800 x[0,4,1] <= 727.0  -t_load[4] + 800 x[0,4,2] <= 730.0
 -t_load[5] + 800 x[0,5,1] <= 786.0  -t_load[5] + 800 x[0,5,2] <= 789.0

## Solution

In [16]:
JuMP.optimize!(model) # Old syntax: status = JuMP.solve(model)

In [17]:
JuMP.value.(x)

MathOptInterface.ResultIndexBoundsError{MathOptInterface.VariablePrimal}: Result index of attribute MathOptInterface.VariablePrimal(1) out of bounds. There are currently 0 solution(s) in the model.

In [18]:
sol_x = Dict{Int, Array}()
for p=1:P
    for i=1:C
        for q=1:Q
            if JuMP.value.(x)[p,i,q] == 1
                if haskey(sol_x, q) == false
                    sol_x[q] = Array{NamedTuple{(:pos, :cont),Tuple{Int64,Int64}}, 1}()
                end
                push!(sol_x[q], (pos=p, cont=i))
            end                
        end
    end
end

sol_x[1]

MathOptInterface.ResultIndexBoundsError{MathOptInterface.VariablePrimal}: Result index of attribute MathOptInterface.VariablePrimal(1) out of bounds. There are currently 0 solution(s) in the model.

In [19]:
sol_x[2]

KeyError: KeyError: key 2 not found

In [20]:
sol_t = JuMP.value.(t_load)

MathOptInterface.ResultIndexBoundsError{MathOptInterface.VariablePrimal}: Result index of attribute MathOptInterface.VariablePrimal(1) out of bounds. There are currently 0 solution(s) in the model.

In [21]:
sol_y = Dict{Int, Array}()
for t=0:H-1
    for j=1:J
        for q=1:Q
            if JuMP.value.(y)[j,q,t] == 1
                if haskey(sol_y, q) == false
                    sol_y[q] = Array{NamedTuple{(:time, :bay),Tuple{Int64,Int64}}, 1}()
                end
                push!(sol_y[q], (time=t, bay=j))
            end                
        end
    end
end

sol_y[1]

UndefVarError: UndefVarError: y not defined

In [22]:
sol_y[2]

KeyError: KeyError: key 2 not found